<link rel='stylesheet' href='../assets/css/main.css'/>

# Lab 4.7 : Spark UDF


### Overview
Using User Defined Functions with Spark Dataframes

### Depends On 
None

### Run time
15-20 mins


In [ ]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()


## STEP 1: Load MT Cars data

In [ ]:
mtcarsDF = spark.read.csv("../data/cars/mtcars_header.csv", header="true")
print(mtcarsDF)



## STEP 2 : Inspecting The Dataframe

**==> Print the schema of data frame**     


In [ ]:
mtcarsDF.printSchema()

In [ ]:
mtcarsDF.show()

In [ ]:
mpgDF = mtcarsDF.select("model", mtcarsDF.mpg.cast("float"))
mpgDF.show()

In [ ]:
#Register dataframe as a table
mpgDF.createOrReplaceTempView("mpgDF")
print("created mpgDF temp table")


## STEP 3 : Define the UDF

**==> Create a UDF to convert mpg to kml i.e. kilometers/litre**

In [ ]:
## This is a standard python function

def mpg_to_kml(mpg):
    kml = mpg * 1.6 / 3.7854
    return kml

In [ ]:
#register the UDF with Spark SQL
from pyspark.sql.types import FloatType

spark.udf.register("MPG_to_KmPerLiter", mpg_to_kml, FloatType())

In [ ]:

# Use the UDF in SQL

resultDF = spark.sql("select model, MPG_to_KmPerLiter(mpg) from mpgDF")
resultDF.show()

## Step 5 : Use the UDF in DataFrame


In [ ]:
from pyspark.sql.functions import udf

## create a UDF function
mpg_to_kml_udf = udf(mpg_to_kml, FloatType())

## Use udf function
resultsDF = mpgDF.select("model", mpg_to_kml_udf("mpg").alias("km_per_liter"))
resultsDF.show()

